In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
import numpy as np
import pandas as pd
import scanpy as sc
import anndata
import bbknn
import os
from scipy import sparse
import matplotlib.pyplot as plt
# from scanpy_base_moudle_update2 import *
# import scrublet as scr
import datetime
import harmonypy as hm

sc.settings.verbosity = 3
#sc.logging.print_versions()
# 设置图片的分辨率以及其他样式
sc.settings.set_figure_params(dpi=150, figsize = (4, 3), fontsize=12)

import matplotlib.font_manager
flist = matplotlib.font_manager.get_fontconfig_fonts()
names = [matplotlib.font_manager.FontProperties(fname=fname).get_name() for fname in flist]
print(names)

params={
        #'font.style':'italic',
        'font.weight':'normal',    #or 'blod'
        }
plt.rcParams.update(params)

plt.rcParams['font.family']='Arial'

In [ ]:
adata = sc.read('/home/wangyue/basic-calculation_data/c_Project_outputs/qilu_CRSwNP/EPI.h5ad')
adata

In [ ]:
sc.pl.umap(adata, color=['S100A8','IL1B'], frameon=False)

In [ ]:
sc.settings.set_figure_params(dpi=600, figsize = (4, 4), fontsize=7)
sc.pl.umap(adata, color=['annotation'], add_outline=True, outline_width = (0.2, 0.05), palette="tab20", frameon=False)
sc.pl.umap(adata, color=['tissue'], frameon=False, title='')
sc.pl.umap(adata, color=['Health'], frameon=False, title='')

In [ ]:
adata

In [ ]:

E01_index = adata.obs.loc[adata.obs["tissue"].isin(['inferior turbinate']), :].index

E02_index = adata.obs.loc[adata.obs["tissue"].isin(['middle turbinate']), :].index

E03_index = adata.obs.loc[adata.obs["tissue"].isin(['polyp']), :].index

E04_index = adata.obs.loc[adata.obs["Health"].isin(['healthy control']), :].index

In [ ]:
adata.obs['fig2_barplot'] = 'Rhi-IT'
adata.obs.at[E02_index,'fig2_barplot']='Rhi-MT'
adata.obs.at[E03_index,'fig2_barplot']='Rhi-NP'
adata.obs.at[E04_index,'fig2_barplot']='HC-IT'

In [ ]:
sc.settings.set_figure_params(dpi=600, figsize = (4, 4), fontsize=7)
sc.pl.umap(adata, color=['fig2_barplot'], frameon=False, title='')

In [ ]:
sc.settings.set_figure_params(dpi=150, figsize = (4, 4), fontsize=7)
sc.pl.umap(adata, color=['LYZ','PGC','TFF2','MUC6','MUC5AC','LTF'], frameon=False)

In [ ]:
sc.pl.umap(adata, color=['fig2_barplot'], groups = ['HC-IT','Rhi-IT'], frameon=False, title ='')

In [ ]:
sc.pl.umap(adata, color=['fig2_barplot'], groups = ['Rhi-MT','Rhi-NP'], frameon=False, title ='')

In [ ]:
adata.uns['annotation_colors']

In [ ]:
adata.obs['annotation'].cat.categories

In [ ]:
Groups_tab_1 = pd.crosstab(index=adata.obs['fig2_barplot'],  # Make a crosstab
                        columns=adata.obs['annotation'], margins=True)               # Name the count column
MyTab_1= Groups_tab_1.div(Groups_tab_1["All"], axis=0)
MyTab2_1 = MyTab_1.drop(columns="All")
MyTab2_1 = MyTab2_1.drop(index="All")

MyTab2_1 = MyTab2_1.T
order = ['HC-IT','Rhi-IT','Rhi-MT','Rhi-NP']
MyTab2_1 = MyTab2_1[order]
MyTab2_1 = MyTab2_1.T
#categories = IMM_group[::-1]

MyTab2_1.columns = pd.CategoricalIndex(MyTab2_1.columns.values)

# Sort the columns (axis=1) by the new categorical ordering
MyTab2_1 = MyTab2_1.sort_index(axis=1)

ax = MyTab2_1.plot.bar(
            figsize=(3.7,5),
            stacked=True,
            edgecolor = '#000000',
            linewidth=0.4,
            width=0.8, 
            fontsize=15,
            color={"C01-E01-Basal resting": "#1f77b4",
                   'C01-E02-Basal cycling': "#aec7e8",
                   'C01-E03-Club': "#ffbb78", 
                   'C01-E04-Goblet': "#98df8a", 
                   'C01-E05-SMG': "#ff9896",
                   'C01-E06-MUC5B+SMG': "#c5b0d5",
                   'C01-E07-MMP7+SMG': "#8c564b",
                   'C01-E08-PRB1+SMG': "#e377c2",
                   'C01-E09-Ionocyte': "#7f7f7f",
                   'C01-E10-Ciliated': "#bcbd22",
                   'C01-E11-S100A8+APC': "#17becf",
                   'C01-E12-KRT14+EMT cell': "#9edae5"}
             )


plt.title("", fontsize=12)
plt.ylabel("Fraction of cells", fontsize=15)
plt.xlabel("", fontsize=12)
plt.ylim=1.0

#plt.gca().get_legend().remove() #remove legend
# plt.legend(categories, loc='center left', bbox_to_anchor=(1, 0.6), fontsize=12)
# plt.savefig('Proportion of clusters accross organs.png')
# 去除刻度
#plt.xticks([])
#plt.yticks([])
# ax.tick_params(bottom=False, top=False, left=False, right=False)
handles, labels = ax.get_legend_handles_labels()
ax.legend(reversed(handles), reversed(labels), loc='center left', bbox_to_anchor=(1, 0.6), fontsize=8)
plt.grid(False)

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
plt.show()

In [ ]:
MyTab2_1.T

In [ ]:
# create a dataset
height = [0.003965, 0.037754, 0.001229, 0.019835]
bars = (' ', ' ', ' ', ' ')
x_pos = np.arange(len(bars))

fig, ax = plt.subplots(figsize=(4, 5))
# Create bars with different colors
plt.bar(x_pos, height, color=['blue', 'orange', 'green', 'red'],edgecolor = '#000000',)

# Create names on the x-axis
plt.xticks(x_pos, bars, fontsize = 30)
plt.yticks([0.0, 0.01, 0.02, 0.03, 0.04 ], fontsize = 20)

plt.grid(False)

print(ax.axis())
#ax.axis([-0.54, 2.5400000000000005, 0.0, 1])

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.show()

In [ ]:
import cosg as cosg
import time
sc.settings.set_figure_params(dpi=600, figsize = (4, 3), fontsize=15)
sc.pl.rank_genes_groups_dotplot(adata,groupby='annotation',
                                cmap='Spectral_r',
                                 standard_scale='var',
                                       n_genes=3,key='cosg')

In [ ]:
marker_genes_dict = {
    'C01-E02-Basal cycling': ['MKI67','TOP2A'], # E1
    'C01-E01-Basal resting': ['KRT15','MMP10'], # E2
    'C01-E11-S100A8+APC': ['LY6D','IL1RN','S100A8','SPRR1B','LYPD3'], # E3
    'C01-E03-Club': ['SERPINB3','GABRP'], # E4
    'C01-E12-KRT14+EMT cell': ['MYLK','MEG3'], # E6
    'C01-E10-Ciliated': ['RSPH1','SNTN',], # E6
    'C01-E06-MUC5B+SMG': ['MUC5B','BPIFB2'], # E9
    'C01-E04-Goblet': ['LYPD2','MUC5AC'], # E10
    'C01-E07-MMP7+SMG': ['MMP7','RARRES1'], # E13
    'C01-E09-Ionocyte': ['ASCL3','CLCNKA'], # E11
    'C01-E08-PRB1+SMG': ['PRB1','PRB3'], # E12
    'C01-E05-SMG': ['AZGP1','LYZ'], # E12
}

In [ ]:
from matplotlib import cm, colors
import colorcet as cc

mymap = colors.LinearSegmentedColormap.from_list('my_colormap', cc.CET_CBL2)
mymap

In [ ]:
adata.uns['annotation_colors']

In [ ]:
categeroies_B = adata.obs['annotation'].cat.categories.to_list()
adata_list = []

for i in categeroies_B:
    cat_index = adata.obs.loc[adata.obs["annotation"].isin([i]), :].index
    adata_cat = adata[cat_index, :]
    adata_cat = sc.pp.subsample(adata_cat, n_obs=100, random_state=1, copy=True)
    adata_list.append(adata_cat)
    
adata_B_conca = adata_list[0].concatenate(adata_list[1:12],
                                   join='outer', batch_categories=categeroies_B)
adata_B_conca

In [ ]:
adata_B_conca.uns['annotation_colors'] = adata.uns['annotation_colors']

In [ ]:
adata_B_conca

In [ ]:
sc.settings.set_figure_params(dpi=400, fontsize=15)
ax = sc.pl.heatmap(adata_B_conca, 
                   marker_genes_dict, 
                   groupby='annotation',
                   cmap=mymap, 
                   show_gene_labels=False,
                   figsize=(4.5,9),
                   dendrogram=True,
                   swap_axes=False,
                   standard_scale='var'
                  ) 

In [ ]:
from matplotlib import cm, colors

colors2 = mymap(np.linspace(0, 0.96, 128)) # 30%-100%
colorsComb = np.vstack([colors2])
mymap = colors.LinearSegmentedColormap.from_list('my_colormap', colorsComb)
mymap

In [ ]:
sc.settings.set_figure_params(dpi=400, fontsize=15)
ax = sc.pl.heatmap(adata_B_conca, 
                   marker_genes_dict, 
                   groupby='annotation',
                   cmap=mymap, 
                   show_gene_labels=False,
                   figsize=(4.5,9),
                   dendrogram=True,
                   swap_axes=False,
                   standard_scale='var'
                  ) 

In [ ]:
adata_B_conca

In [ ]:
adata.obs['annotation'].cat.categories

In [ ]:
adata = sc.read('/mnt/data2/Datasets/Human_non_intestine_datasets/Qilu_Otorhinolaryngology_surgery_data/dataset_output/EPI.h5ad')
adata

In [ ]:
adata = sc.AnnData(X=adata.raw.X, var=adata.raw.var, obs = adata.obs)

In [ ]:
categeroies_B = adata.obs['annotation'].cat.categories.to_list()
adata_list = []

for i in categeroies_B:
    if i == 'C01-E11-S100A8+APC':
        cat_index = adata.obs.loc[adata.obs["annotation"].isin([i]), :].index
        adata_cat = adata[cat_index, :]
        adata_cat = sc.pp.subsample(adata_cat, n_obs=1000, random_state=1, copy=True)
        adata_list.append(adata_cat) 
    else:
        cat_index = adata.obs.loc[adata.obs["annotation"].isin([i]), :].index
        adata_cat = adata[cat_index, :]
        adata_cat = sc.pp.subsample(adata_cat, n_obs=500, random_state=1, copy=True)
        adata_list.append(adata_cat)
    
adata_B_conca = adata_list[0].concatenate(adata_list[1:12],
                                   join='outer', batch_categories=categeroies_B)
adata_B_conca

In [ ]:
sc.pp.filter_genes(adata_B_conca, min_cells=10)
adata_B_conca

In [ ]:
E01_index = adata_B_conca.obs.loc[adata_B_conca.obs["annotation"].isin(['C01-E11-S100A8+APC']), :].index

adata_B_conca.obs['DEG'] = 'Other'
adata_B_conca.obs.at[E01_index,'DEG']='C01-E11-S100A8+APC'

In [ ]:
adata_B_conca

In [ ]:
adata_B_conca.to_df().T

In [ ]:
adata_B_conca.to_df().T.to_csv('/mnt/data2/Datasets/Human_non_intestine_datasets/Qilu_Otorhinolaryngology_surgery_data/dataset_output/csv/LY6D_vs_other_matrix.csv')

In [ ]:
adata_B_conca.obs.loc[:,['DEG']]

In [ ]:
adata_B_conca.obs.loc[:,['DEG']].to_csv('/mnt/data2/Datasets/Human_non_intestine_datasets/Qilu_Otorhinolaryngology_surgery_data/dataset_output/csv/LY6D_vs_other_meta.csv')

In [ ]:
sc.settings.set_figure_params(dpi=150, figsize = (4, 4), fontsize=12)

from matplotlib import cm, colors
import colorcet as cc

mymap = colors.LinearSegmentedColormap.from_list('my_colormap', cc.CET_L20)

colors2 = mymap(np.linspace(0.2, 1, 128)) # 30%-100%
colors3 = plt.cm.Greys_r(np.linspace(0.8,0.9,5))
colorsComb = np.vstack([colors3, colors2])
mymap = colors.LinearSegmentedColormap.from_list('my_colormap', colorsComb)

sc.pl.umap(adata, color=['KRT5'], frameon=False, color_map = mymap)

In [ ]:
sc.settings.set_figure_params(dpi=300, figsize = (4, 4), fontsize=12)

from matplotlib import cm, colors
import colorcet as cc

mymap = colors.LinearSegmentedColormap.from_list('my_colormap', cc.CET_L20)

colors2 = mymap(np.linspace(0.2, 1, 128)) # 30%-100%
colors3 = plt.cm.Greys_r(np.linspace(0.8,0.9,5))
colorsComb = np.vstack([colors3, colors2])
mymap = colors.LinearSegmentedColormap.from_list('my_colormap', colorsComb)

sc.pl.umap(adata, color=['CXCL1','CXCL2','CXCL3','CXCL5'], frameon=False, color_map = mymap)
sc.pl.umap(adata, color=['CXCL6','CXCL8','PPBP','S100P'], frameon=False, color_map = mymap)                         
sc.pl.umap(adata, color=['CXCL9','CXCL10','CXCL11','EREG'], frameon=False, color_map = mymap)

In [ ]:
sc.pl.umap(adata, color=['LYPD3','IDO1','MUC5AC','S100P'], frameon=False, color_map = mymap)                         
sc.pl.umap(adata, color=['TFF2','LYZ','CD24','EREG'], frameon=False, color_map = mymap)
sc.pl.umap(adata, color=['SCGB1A1','SCGB3A1','SCGB3A2'], frameon=False, color_map = mymap)

In [ ]:
pd.DataFrame(adata.uns['cosg']['names'])['C01-E06-MUC5B+SMG'].head(50)

In [ ]:
pd.DataFrame(adata.uns['cosg']['names'])['C01-E11-S100A8+APC'].head(50)

In [ ]:
adata.obs['annotation'].cat.categories

In [ ]:
tf_GENES = '/mnt/data/project/scenic/auxilliaries/lambert2018_c.txt'
amps_pd = pd.read_table(tf_GENES)
amps_list = list(amps_pd['Gene_name'])
amps_list = [x for x in amps_list if x in adata.raw.var_names]
adata_c = sc.AnnData(X=adata.raw.X, var=adata.raw.var, obs = adata.obs)
adata_c = adata_c[:,amps_list]

## TFs

import cosg as cosg
import time
t0= time.clock()
cosg.cosg(adata_c,
    key_added='cosg',
        mu=1,
        n_genes_user=50,
               groupby='annotation')
runtime_cosg = time.clock() - t0

sc.settings.set_figure_params(dpi=200, figsize = (4, 4), fontsize=15)
sc.pl.rank_genes_groups_dotplot(adata_c,groupby='annotation',
                                cmap='Spectral_r',
                                 standard_scale='var',
                                       n_genes=3,key='cosg')

In [ ]:
adata = sc.read('/mnt/data2/Datasets/Human_non_intestine_datasets/Qilu_Otorhinolaryngology_surgery_data/dataset_output/EPI.h5ad')
adata

In [ ]:
adata.obs['tissue'].cat.categories

In [ ]:
E_index = adata.obs.loc[adata.obs["tissue"].isin(['inferior turbinate']), :].index
adata = adata[E_index, :]
adata

In [ ]:
adata.obs['annotation'].cat.categories

In [ ]:
E_index = adata.obs.loc[adata.obs["annotation"].isin(['C01-E03-Club','C01-E11-S100A8+APC']), :].index
adata = adata[E_index, :]
adata

In [ ]:
adata = sc.AnnData(X=adata.raw.X, var=adata.raw.var, obs = adata.obs)

In [ ]:
categeroies_B = adata.obs['Health'].cat.categories.to_list()
adata_list = []

for i in categeroies_B:
    cat_index = adata.obs.loc[adata.obs["Health"].isin([i]), :].index
    adata_cat = adata[cat_index, :]
    adata_cat = sc.pp.subsample(adata_cat, n_obs=1000, random_state=1, copy=True)
    adata_list.append(adata_cat)
    
adata_B_conca = adata_list[0].concatenate(adata_list[1:2],
                                   join='outer', batch_categories=categeroies_B)
adata_B_conca

In [ ]:
sc.pp.filter_genes(adata_B_conca, min_cells=10)
adata_B_conca

In [ ]:
adata_B_conca.to_df().T.to_csv('/mnt/data2/Datasets/Human_non_intestine_datasets/Qilu_Otorhinolaryngology_surgery_data/dataset_output/csv/Club_CRS_vs_Club_HC.csv')

In [ ]:
adata_B_conca.obs.loc[:,['Health']]
adata_B_conca.obs.loc[:,['Health']].to_csv('/mnt/data2/Datasets/Human_non_intestine_datasets/Qilu_Otorhinolaryngology_surgery_data/dataset_output/csv/Club_CRS_vs_Club_HC_meta.csv')

**GSVA**

In [ ]:
adata_B_conca

In [ ]:
fn = '/mnt/data/Project2021/Gut_immune_surveillance/ref_geneset/c2.cp.reactome.v7.2.symbols.gmt'

with open(fn) as f:
    sets_raw = f.readlines()
sets_proc = [x.split('\n')[0] for x in sets_raw]
sets_proc = [x.split('\t') for x in sets_proc]

path_name_list = []
gene_ids_list = []

for x in sets_proc:
    path_name = x[0]
    gene_ids=x[2:]
    
    path_name_list.append(path_name)
    gene_ids_list.append(gene_ids)

In [ ]:
dataframe_GSVA = pd.DataFrame(index=adata_B_conca.obs.index, columns=path_name_list)
dataframe_GSVA

In [ ]:
# 1554 pathway,time cost 856.7754094600677 s
import time

time_start = time.time()

j = 0
for i in path_name_list:
    score_name = i
    genesets = gene_ids_list[j]
    print(len(genesets))
    genesets = [x for x in genesets if x in adata_B_conca.var_names]
    print(len(genesets))
    print(j)
    j = j+1
    
    sc.tl.score_genes(adata_B_conca, genesets, 
                      ctrl_size=len(genesets), 
                      gene_pool=None, 
                      n_bins=25, 
                      score_name=score_name, 
                      random_state=0, 
                      copy=False, 
                      use_raw=None)
    dataframe_GSVA[score_name] = adata_B_conca.obs[score_name]

time_end = time.time()    #结束计时

time_c= time_end - time_start   #运行所花时间
print('time cost', time_c, 's')

In [ ]:
dataframe_GSVA

In [ ]:
dataframe_GSVA = dataframe_GSVA.applymap(lambda x: x +1)

In [ ]:
import seaborn as sns
sns.kdeplot(dataframe_GSVA['REACTOME_REMOVAL_OF_THE_FLAP_INTERMEDIATE_FROM_THE_C_STRAND'], shade = True)

In [ ]:
index = dataframe_GSVA.index.to_frame(index=True, name='barcodes')
columns = dataframe_GSVA.columns.to_frame(index=True, name='pathway')

# 将dataframe_GSVA读取为Anndata
anndata_GSVA = sc.AnnData(X=dataframe_GSVA.values, var=columns, obs = index)
anndata_GSVA

In [ ]:
anndata_GSVA.raw = anndata_GSVA
anndata_GSVA.obs['annotation']=adata_B_conca.obs['annotation']
anndata_GSVA.obs

In [ ]:
sc.pp.scale(anndata_GSVA, max_value=10)

In [ ]:
# 所有器官，bulk级别
def adata_diff(adata_test):
    
    sc.settings.set_figure_params(dpi=100, figsize = (4, 3), fontsize=6)
    sc.tl.rank_genes_groups(adata_test, 'annotation', method='wilcoxon')
    #sc.pl.rank_genes_groups(adata_test, n_genes=25, sharey=False)
    pd.DataFrame(adata_test.uns['rank_genes_groups']['names']).to_csv('/mnt/data2/Datasets/Human_non_intestine_datasets/Qilu_Otorhinolaryngology_surgery_data/dataset_output/csv/'+ dirfile)

    #sc.get.rank_genes_groups_df(adata_test, group="1-Duodenum")
    
    sc.settings.set_figure_params(dpi=150, figsize = (4, 3), fontsize=6)

    sc.pl.rank_genes_groups_heatmap(adata_test, 
                                    n_genes=20, 
                                    use_raw=False, 
                                    swap_axes=True, 
                                    vmin=-2, 
                                    vmax=2, 
                                    cmap='bwr', 
                                    # layer='scaled', 
                                    figsize=(10,10),
                                    # categories_order = categories_order,
                                    show=True,
                                    dendrogram=False,
                                    show_gene_labels=True,
                                    # save = '_hotmap3.png'
                                   )
    
    sc.settings.set_figure_params(dpi=150, figsize = (4, 3), fontsize=20)
    mp1 = sc.pl.rank_genes_groups_matrixplot(adata_test, 
                                       n_genes=6, 
                                       use_raw=False, 
                                       dendrogram=False,
                                       return_fig=True,
                                       vmin=-1.7, 
                                       vmax=1.7,
                                       #categories_order = categories_order,
                                       cmap='bwr')
    mp1.style(edge_color='white',cmap = 'bwr',edge_lw=1.0).show()
    
    mp1 = sc.pl.rank_genes_groups_matrixplot(adata_test, 
                                       n_genes=6, 
                                       use_raw=False, 
                                       dendrogram=False,
                                       return_fig=True,
                                       vmin=-1.7, 
                                       vmax=1.7,
                                       #categories_order = categories_order,
                                       cmap='bwr')
    mp1.style(edge_color='white',cmap = 'RdYlBu_r',edge_lw=1.0).show()   
    
    mp1 = sc.pl.rank_genes_groups_matrixplot(adata_test, 
                                       n_genes=6, 
                                       use_raw=False, 
                                       dendrogram=False,
                                       return_fig=True,
                                       vmin=-1.7, 
                                       vmax=1.7,
                                       #categories_order = categories_order,
                                       cmap='bwr')
    mp1.style(edge_color='white',cmap = 'bwr',edge_lw=1.0).show()       

    
    return adata_test

In [ ]:
adata_test = anndata_GSVA
dirfile = 'LY6DClub_rank_gsva_reactome_20211210.csv'
adata_diff(adata_test)

In [ ]:
adata_test.obs['annotation'].cat.categories

In [ ]:
pd_Proximal_SI = sc.get.rank_genes_groups_df(adata_test, group="C01-E11-S100A8+APC")
pd_Proximal_SI

In [ ]:
pd_Proximal_SI.to_csv('/mnt/data2/Datasets/Human_non_intestine_datasets/Qilu_Otorhinolaryngology_surgery_data/dataset_output/csv/LY6D_Club_GSVA.csv')

In [ ]:
REACTOME_REGULATION_OF_TLR_BY_ENDOGENOUS_LIGAND
REACTOME_NEUTROPHIL_DEGRANULATION
REACTOME_ANTIGEN_PROCESSING_CROSS_PRESENTATION
REACTOME_INTERLEUKIN_1_SIGNALING
REACTOME_FCERI_MEDIATED_NF_KB_ACTIVATION
REACTOME_INTERLEUKIN_1_FAMILY_SIGNALING
REACTOME_HOMOLOGOUS_DNA_PAIRING_AND_STRAND_EXCHANGE
REACTOME_ROLE_OF_PHOSPHOLIPIDS_IN_PHAGOCYTOSIS
REACTOME_COLLAGEN_BIOSYNTHESIS_AND_MODIFYING_ENZYMES
REACTOME_RESOLUTION_OF_D_LOOP_STRUCTURES
REACTOME_COHESIN_LOADING_ONTO_CHROMATIN
REACTOME_RESOLUTION_OF_D_LOOP_STRUCTURES_THROUGH_SYNTHESIS_DEPENDENT_STRAND_ANNEALING_SDSA_

In [ ]:
# create a dataset
height = [40.038708, 39.046436, 35.79242, 34.380756, 33.42309, 32.852737, 
          23.435366, 25.43655, 25.552229, 25.591013, 25.835993, 28.000149]
bars = (' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ')
x_pos = np.arange(len(bars))

fig, ax = plt.subplots(figsize=(5, 5))
# Create bars with different colors
plt.bar(x_pos, height, color=['#fa8484','#fa8484','#fa8484','#fa8484','#fa8484','#fa8484',
                              '#75afff','#75afff','#75afff','#75afff','#75afff','#75afff'])

# Create names on the x-axis
plt.xticks(x_pos, bars, fontsize = 30)
plt.yticks([10, 20, 30, 40, 50 ], fontsize = 20)

plt.grid(False)

print(ax.axis())
#ax.axis([-0.54, 2.5400000000000005, 0.0, 1])

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.show()

In [ ]:
sc.settings.set_figure_params(dpi=300, figsize = (4, 4), fontsize=12)

from matplotlib import cm, colors
import colorcet as cc

mymap = colors.LinearSegmentedColormap.from_list('my_colormap', cc.CET_L20)

colors2 = mymap(np.linspace(0.2, 1, 128)) # 30%-100%
colors3 = plt.cm.Greys_r(np.linspace(0.8,0.9,5))
colorsComb = np.vstack([colors3, colors2])
mymap = colors.LinearSegmentedColormap.from_list('my_colormap', colorsComb)

sc.pl.umap(adata, color=['EREG'], frameon=False, color_map = mymap, size=3)
sc.pl.umap(adata, color=['IL1RN'], frameon=False, color_map = mymap, size=3)
sc.pl.umap(adata, color=['HLA-DQA2'], frameon=False, color_map = mymap, size=3)

In [ ]:
import os, glob, re, pickle
from functools import partial
from collections import OrderedDict
import operator as op
from cytoolz import compose

import pandas as pd
import seaborn as sns
import numpy as np
import scanpy as sc
import anndata as ad
import matplotlib as mpl
import matplotlib.pyplot as plt
import loompy as lp

from pyscenic.export import export2loom, add_scenic_metadata
from pyscenic.utils import load_motifs
from pyscenic.transform import df2regulons
from pyscenic.aucell import aucell
from pyscenic.binarization import binarize
from pyscenic.rss import regulon_specificity_scores
from pyscenic.plotting import plot_binarization, plot_rss

from IPython.display import HTML, display

import matplotlib.font_manager
flist = matplotlib.font_manager.get_fontconfig_fonts()
names = [matplotlib.font_manager.FontProperties(fname=fname).get_name() for fname in flist]
print(names)

params={
        #'font.style':'italic',
        'font.weight':'normal',    #or 'blod'
        }
plt.rcParams.update(params)

plt.rcParams['font.family']='Arial'

In [ ]:
tf_GENES = '/mnt/data/project/scenic/auxilliaries/lambert2018_c.txt'
amps_pd = pd.read_table(tf_GENES)
amps_list = list(amps_pd['Gene_name'])
adata = sc.read('/mnt/data2/Datasets/Human_non_intestine_datasets/Qilu_Otorhinolaryngology_surgery_data/dataset_output/EPI.h5ad')
amps_list = [x for x in amps_list if x in adata.raw.var_names]
adata_c = sc.AnnData(X=adata.raw.X, var=adata.raw.var, obs = adata.obs)
adata_c = adata_c[:,amps_list]
adata_c

In [ ]:
sc.pp.filter_genes(adata_c, min_cells=5)
sc.pp.scale(adata_c, max_value=1)
auc_mtx = adata_c.to_df()
# 需要保证后续加上一个数值后所有的数均大于0
np.min(auc_mtx.min().to_list())

In [ ]:
auc_mtx = auc_mtx.applymap(lambda x: x +2)

In [ ]:
# Calculate RSS
rss = regulon_specificity_scores(auc_mtx, adata_c.obs.annotation)
rss.head()

In [ ]:
rss.T.to_csv('/mnt/data2/Datasets/Human_non_intestine_datasets/Qilu_Otorhinolaryngology_surgery_data/dataset_output/SCENIC/rss_epi.csv')

In [ ]:
sc.settings.set_figure_params(dpi=400)

In [ ]:
from adjustText import adjust_text

cats = list(rss.index)

fig = plt.figure(figsize=(15,14)) # 长/宽
for c,num in zip(cats, range(1,len(cats)+1)):
    x=rss.T[c]
    ax = fig.add_subplot(3,4,num) # 设置3行4列的图
    plot_rss(rss, c, top_n=20, max_n=None, ax=ax)
    ax.set_ylim( x.min()-(x.max()-x.min())*0.05 , x.max()+(x.max()-x.min())*0.05 )
    plt.grid(False)
    for t in ax.texts:
        t.set_fontsize(12)
    ax.tick_params(labelsize=15)
    ax.set_ylabel('')
    ax.set_xlabel('')
    adjust_text(ax.texts, autoalign='xy', ha='right', va='bottom', arrowprops=dict(arrowstyle='-',color='lightgrey'), precision=0.001 )
 
fig.text(0.5, 0.0, 'Regulon', ha='center', va='center', size='x-large')
fig.text(0.00, 0.5, 'Regulon specificity score (RSS)', ha='center', va='center', rotation='vertical', size='x-large')
plt.tight_layout()
plt.rcParams.update({
    'figure.autolayout': True,
        'figure.titlesize': 'large' ,
        'axes.labelsize': 'medium',
        'axes.titlesize':'large',
        'xtick.labelsize':'medium',
        'ytick.labelsize':'medium'
        })
# plt.savefig("PBMC10k_cellType-RSS-top5.pdf", dpi=600, bbox_inches = "tight")

plt.show()

In [ ]:
from adjustText import adjust_text

cats = list(rss.index)

fig = plt.figure(figsize=(15,14)) # 长/宽
for c,num in zip(cats, range(1,len(cats)+1)):
    x=rss.T[c]
    ax = fig.add_subplot(3,4,num) # 设置3行4列的图
    plot_rss(rss, c, top_n=20, max_n=None, ax=ax)
    ax.set_ylim( x.min()-(x.max()-x.min())*0.05 , x.max()+(x.max()-x.min())*0.05 )
    plt.grid(False)
    for t in ax.texts:
        t.set_fontsize(0)
    ax.tick_params(labelsize=15)
    ax.set_ylabel('')
    ax.set_xlabel('')
    # adjust_text(ax.texts, autoalign='xy', ha='right', va='bottom', arrowprops=dict(arrowstyle='-',color='lightgrey'), precision=0.001 )
 
fig.text(0.5, 0.0, 'Regulon', ha='center', va='center', size='x-large')
fig.text(0.00, 0.5, 'Regulon specificity score (RSS)', ha='center', va='center', rotation='vertical', size='x-large')
plt.tight_layout()
plt.rcParams.update({
    'figure.autolayout': True,
        'figure.titlesize': 'large' ,
        'axes.labelsize': 'medium',
        'axes.titlesize':'large',
        'xtick.labelsize':'medium',
        'ytick.labelsize':'medium'
        })
# plt.savefig("PBMC10k_cellType-RSS-top5.pdf", dpi=600, bbox_inches = "tight")

plt.show()

In [ ]:
# Basal resting
E01_index = adata_c.obs.loc[adata_c.obs["tissue"].isin(['inferior turbinate']), :].index
# Basal cycling
E02_index = adata_c.obs.loc[adata_c.obs["tissue"].isin(['middle turbinate']), :].index
# Club
E03_index = adata_c.obs.loc[adata_c.obs["tissue"].isin(['polyp']), :].index
# Health
E04_index = adata_c.obs.loc[adata_c.obs["Health"].isin(['healthy control']), :].index

adata_c.obs['fig2_barplot'] = 'Rhi-IT'
adata_c.obs.at[E02_index,'fig2_barplot']='Rhi-MT'
adata_c.obs.at[E03_index,'fig2_barplot']='Rhi-NP'
adata_c.obs.at[E04_index,'fig2_barplot']='HC-IT'

E_index = adata_c.obs.loc[~adata_c.obs["fig2_barplot"].isin(['HC-IT']), :].index
adata_c = adata_c[E_index, :]
adata_c

In [ ]:
phenotype = 'tissue'
annotation = 'annotation'

ann_list = adata_c.obs[annotation].to_list()
phenotype_list = adata_c.obs[phenotype].to_list()
Ann=[phenotype_list[i]+ '_' + ann_list[i] for i in range(min(len(phenotype_list),len(ann_list)))]
adata_c.obs['Ann_for_tree'] = Ann

In [ ]:
adata_c.obs['Ann_for_tree'] = adata_c.obs['Ann_for_tree'].astype('category')

In [ ]:
sc.pp.scale(adata_c, zero_center=False)
sc.tl.pca(adata_c, svd_solver='arpack')
sc.tl.dendrogram(adata_c, 'Ann_for_tree')

In [ ]:
sc.settings.set_figure_params(dpi=600, fontsize=3.5,figsize=(5, 2.5))
sc.pl.dendrogram(adata_c, 'Ann_for_tree',save='True')

In [ ]:
sc.settings.set_figure_params(dpi=600, fontsize=3.5,figsize=(5, 2.5))
sc.pl.dendrogram(adata_c, 'Ann_for_tree',save='True')

In [ ]:
aucell_adata = adata_c

In [ ]:
z_var = aucell_adata.uns['dendrogram_Ann_for_tree']['linkage']
categories = aucell_adata.obs['Ann_for_tree'].cat.categories

import scipy
T = scipy.cluster.hierarchy.to_tree(z_var , rd=False )

# Create dictionary for labeling nodes by their IDs
labels=list(categories)
id2name = dict(enumerate(labels))

In [ ]:
# Load required modules
import pandas as pd 
import scipy.spatial
import scipy.cluster
import numpy as np
import json
import matplotlib.pyplot as plt
from functools import reduce

# Create a nested dictionary from the ClusterNode's returned by SciPy
def add_node(node, parent ):
    # First create the new node and append it to its parent's children
    newNode = dict( node_id=node.id, children=[] )
    parent["children"].append( newNode )

    # Recursively add the current node's children
    if node.left: add_node( node.left, newNode )
    if node.right: add_node( node.right, newNode )

# Initialize nested dictionary for d3, then recursively iterate through tree
d3Dendro = dict(children=[], name="Root1")
add_node( T, d3Dendro )


# Label each node with the names of each leaf in its subtree
def label_tree( n ):
    # If the node is a leaf, then we have its name
    if len(n["children"]) == 0:
        leafNames = [ id2name[n["node_id"]] ]
    
    # If not, flatten all the leaves in the node's subtree
    else:
        leafNames = reduce(lambda ls, c: ls + label_tree(c), n["children"], [])

    # Delete the node id since we don't need it anymore and
    # it makes for cleaner JSON
    del n["node_id"]

    # Labeling convention: "&"-separated leaf names
    n["name"] = name = "&".join(sorted(map(str, leafNames)))
    
    return leafNames

label_tree( d3Dendro["children"][0] )

In [ ]:
tree=[]  # tree用于循环差异基因(转录因子)分析
def preorder(root):
    if not root.is_leaf():
        str1=",".join([list(categories)[i] for i in root.left.pre_order()])
        str2=",".join([list(categories)[i] for i in root.right.pre_order()])
        tree.append([str1,str2])
    if root.get_left() is not None:
        preorder(root.get_left())
    if root.get_right() is not None:
        preorder(root.get_right())
preorder(T)
print(tree)

In [ ]:
group_0_name_list = []
group_1_name_list = []
rank_reg_group0_list = []
rank_reg_group1_list = []

for i in tree:
    group0 = i[0].split(',')
    group_0_name_list.append(group0)
    
    group1 = i[1].split(',')
    group_1_name_list.append(group1)
    
    # group = group0 + group1
    
    group0_index = aucell_adata.obs.loc[aucell_adata.obs["Ann_for_tree"].isin(group0), :].index
    group1_index = aucell_adata.obs.loc[aucell_adata.obs["Ann_for_tree"].isin(group1), :].index
    
    # 建立新的dataframe列['group_DEG']，并将group0的index对应的列的值修改为group0
    aucell_adata.obs['group_DEG'] = aucell_adata.obs['Ann_for_tree']
    # 解决categories报错的问题，将原本不存在的类别添加，并删除不再使用的类别
    new_categories = ['group0']
    aucell_adata.obs['group_DEG'].cat.add_categories(new_categories, inplace = True)
    aucell_adata.obs.loc[group0_index,'group_DEG']='group0'
    aucell_adata.obs['group_DEG'].cat.remove_unused_categories(inplace = True)
    
    # 在dataframe列['group_DEG']修改了group0的基础上，将group1的index对应的列的值修改为group1
    new_categories = ['group1']
    aucell_adata.obs['group_DEG'].cat.add_categories(new_categories, inplace = True)
    aucell_adata.obs.loc[group1_index,'group_DEG']='group1'
    aucell_adata.obs['group_DEG'].cat.remove_unused_categories(inplace = True)

    sc.tl.rank_genes_groups(aucell_adata, 
                        'group_DEG', 
                        method='wilcoxon',
                        groups=['group0'],
                        reference='group1')
    result_DEG = sc.get.rank_genes_groups_df(aucell_adata, 
                                             group="group0")
    
    rank_reg_top10_group0 = list(result_DEG['names'])[0:10]
    rank_reg_group0_list.append(rank_reg_top10_group0)
    
    rank_reg_top10_group1 = list(result_DEG['names'])[-10:][::-1] # group1的top10为最后10个reg的倒序.
    rank_reg_group1_list.append(rank_reg_top10_group1)
    
result_reg = pd.DataFrame()
result_reg['group_0_name_list'] = group_0_name_list
result_reg['group_1_name_list'] = group_1_name_list
result_reg['rank_reg_top10_group0'] = rank_reg_group0_list
result_reg['rank_reg_top10_group1'] = rank_reg_group1_list
result_reg

In [ ]:
result_reg.to_csv('/mnt/data2/Datasets/Human_non_intestine_datasets/Qilu_Otorhinolaryngology_surgery_data/dataset_output/csv/epi_DET.csv')

**CellRank**

In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
import numpy as np
import pandas as pd
import scanpy as sc
import anndata
import bbknn
import os
from scipy import sparse
import matplotlib.pyplot as plt
# from scanpy_base_moudle_update2 import *
# import scrublet as scr
import datetime
import harmonypy as hm

sc.settings.verbosity = 3
#sc.logging.print_versions()
# 设置图片的分辨率以及其他样式
sc.settings.set_figure_params(dpi=150, figsize = (4, 3), fontsize=12)

import matplotlib.font_manager
flist = matplotlib.font_manager.get_fontconfig_fonts()
names = [matplotlib.font_manager.FontProperties(fname=fname).get_name() for fname in flist]
print(names)

params={
        #'font.style':'italic',
        'font.weight':'normal',    #or 'blod'
        }
plt.rcParams.update(params)

plt.rcParams['font.family']='Arial'

In [ ]:
adata = sc.read('/mnt/data2/Datasets/Human_non_intestine_datasets/Qilu_Otorhinolaryngology_surgery_data/dataset_output/EPI.h5ad')
adata

In [ ]:
adata.obs["annotation"].cat.categories

In [ ]:
E_index = adata.obs.loc[adata.obs["annotation"].isin(['C01-E01-Basal resting', 'C01-E02-Basal cycling', 'C01-E03-Club',
                                                     'C01-E04-Goblet','C01-E11-S100A8+APC']), :].index
adata = adata[E_index, :]
adata

In [ ]:
# Basal resting
E01_index = adata.obs.loc[adata.obs["tissue"].isin(['inferior turbinate']), :].index
# Basal cycling
E02_index = adata.obs.loc[adata.obs["tissue"].isin(['middle turbinate']), :].index
# Club
E03_index = adata.obs.loc[adata.obs["tissue"].isin(['polyp']), :].index
# Health
E04_index = adata.obs.loc[adata.obs["Health"].isin(['healthy control']), :].index

adata.obs['fig2_barplot'] = 'Rhi-IT'
adata.obs.at[E02_index,'fig2_barplot']='Rhi-MT'
adata.obs.at[E03_index,'fig2_barplot']='Rhi-NP'
adata.obs.at[E04_index,'fig2_barplot']='HC-IT'

In [ ]:
## 注意，diifmap这步会修改Neighbors
adata_diff = adata

sc.tl.diffmap(adata_diff)

sc.pp.neighbors(adata_diff, n_neighbors=20, use_rep='X_diffmap')
sc.tl.draw_graph(adata_diff)

sc.settings.set_figure_params(dpi=150, figsize = (4, 3), fontsize=6)
sc.pl.draw_graph(adata_diff, color='annotation', legend_loc='on data', frameon=False)
sc.pl.draw_graph(adata_diff, color='annotation', frameon=False)
sc.pl.draw_graph(adata_diff, color='fig2_barplot', frameon=False)
#sc.pl.draw_graph(adata_concat, color='batch', frameon=False)

In [ ]:
adata_diff.write('/mnt/data2/Datasets/Human_non_intestine_datasets/Qilu_Otorhinolaryngology_surgery_data/dataset_output/Nasal_epithelium_FA2.h5ad')

In [ ]:
adata_paga = adata_diff

In [ ]:
# 基于PAGA的轨迹推断
sc.tl.paga(adata_paga, groups='annotation')
sc.pl.paga(adata_paga, color=['annotation'], frameon=False)

sc.tl.draw_graph(adata_paga, init_pos='paga')
sc.pl.draw_graph(adata_paga, color=['annotation'], legend_loc='on data', frameon=False)

sc.settings.set_figure_params(dpi=150, figsize = (7, 7), fontsize=6)
sc.pl.paga_compare(
    adata_paga, threshold=0.1, title='', right_margin=0.5, size=10, edge_width_scale=0.5,
    legend_fontsize=12, fontsize=6, frameon=False, edges=True, save=False)

sc.settings.set_figure_params(dpi=150, figsize = (4, 4), fontsize=6)
sc.pl.draw_graph(adata_paga, color='annotation',frameon=False)

In [ ]:
adata_paga.write('/mnt/data2/Datasets/Human_non_intestine_datasets/Qilu_Otorhinolaryngology_surgery_data/dataset_output/Nasal_epithelium_PAGA.h5ad')

In [ ]:
sc.settings.set_figure_params(dpi=300, figsize = (4, 4), fontsize=6)
sc.pl.paga(adata, color=['annotation'], frameon=False, fontsize=0, edge_width_scale = 0.5)

In [ ]:
# 导入包及Figure和font设置
import scvelo as scv
import scanpy as sc
import cellrank as cr
import numpy as np

scv.settings.verbosity = 3
scv.settings.set_figure_params("scvelo")
cr.settings.verbosity = 2

import warnings

warnings.simplefilter("ignore", category=UserWarning)
warnings.simplefilter("ignore", category=FutureWarning)
warnings.simplefilter("ignore", category=DeprecationWarning)

import matplotlib.pyplot as plt
import matplotlib.font_manager
flist = matplotlib.font_manager.get_fontconfig_fonts()
names = [matplotlib.font_manager.FontProperties(fname=fname).get_name() for fname in flist]
print(names)

params={
        #'font.style':'italic',
        'font.weight':'normal',    #or 'blod'
        }
plt.rcParams.update(params)

plt.rcParams['font.family']='Arial'

In [ ]:
adata = sc.read('/mnt/data2/Datasets/Human_non_intestine_datasets/Qilu_Otorhinolaryngology_surgery_data/dataset_output/Nasal_epithelium_PAGA.h5ad')
adata

In [ ]:
adata = sc.AnnData(X=adata.raw.X, 
                   var=adata.raw.var, 
                   obs = adata.obs, 
                   uns=adata.uns,
                   obsm=adata.obsm)

adata

In [ ]:
ax = scv.pl.scatter(adata, basis="draw_graph_fa", c="annotation", legend_loc="right",save=False)

In [ ]:
sc.pp.filter_genes(adata, min_cells=10)
sc.pp.highly_variable_genes(adata)
print(f"This detected {np.sum(adata.var['highly_variable'])} highly variable genes. ")

adata.layers["spliced"] = adata.X
adata.layers["unspliced"] = adata.X

# 这步会在HVG基础上，计算PCA，Neighbors，并添加layers["Ms"]、layers["Mu"]
scv.pp.moments(adata, n_pcs=30, n_neighbors=30)
adata

**Initialize the CytoTRACE kernel**

In [ ]:
from cellrank.tl.kernels import CytoTRACEKernel

ctk = CytoTRACEKernel(adata)

In [ ]:
scv.pl.scatter(
    adata,
    c=["ct_pseudotime", "annotation"],
    basis="draw_graph_fa",
    legend_loc="right",
    color_map="gnuplot2",
)

In [ ]:
sc.pl.violin(adata, keys=["ct_pseudotime"], groupby="annotation", rotation=90)

**Compute & visualize a transition matrix**

In [ ]:
# compute the transition matrix
ctk.compute_transition_matrix(threshold_scheme="soft", nu=0.5)

In [ ]:
# ctk的方向添加至force_directed布局可视化中
ctk.compute_projection(basis="draw_graph_fa")

In [ ]:
# 可视化
scv.pl.velocity_embedding_stream(
    adata, color="annotation", vkey="T_fwd", basis="draw_graph_fa", legend_loc="right",
    density=2, size=30,figsize=(10,10),linewidth=3,arrow_size=1.5,outline_width=None,
)

**Compute macrostates 计算宏状态**

In [ ]:
from cellrank.tl.estimators import GPCCA

g_fwd = GPCCA(ctk)
print(g_fwd)

In [ ]:
g_fwd.compute_schur(n_components=20)
g_fwd.plot_spectrum(real_only=True)

In [ ]:
g_fwd.compute_macrostates(n_states=3, cluster_key="annotation")

g_fwd.plot_macrostates(
    discrete=True, legend_loc="right", size=100, basis="draw_graph_fa"
)

In [ ]:
g_fwd.plot_coarse_T()

In [ ]:
g_fwd.compute_macrostates(n_states=3, cluster_key="annotation")
g_fwd.set_terminal_states_from_macrostates(names=["C01-E11-S100A8+APC",
                                                  "C01-E01-Basal resting",
                                                  "C01-E04-Goblet"])

**Compute fate probabilities 计算命运概率**

In [ ]:
g_fwd.compute_absorption_probabilities()
g_fwd.plot_absorption_probabilities(same_plot=False, size=50, basis="draw_graph_fa")

**Compute lineage drivers**

In [ ]:
drivers = g_fwd.compute_lineage_drivers(return_drivers=True)
drivers

In [ ]:
drivers.sort_values(by = 'C01-E11-S100A8+APC_corr', ascending=False)

In [ ]:
g_fwd.plot_lineage_drivers("C01-E11-S100A8+APC", n_genes=8, basis="draw_graph_fa")

**Gene expression trends**

In [ ]:
np.min(list(adata.obs["ct_pseudotime"]))

In [ ]:
np.where(adata.obs["ct_pseudotime"] == 0)[0][0]

In [ ]:
# compue DPT, starting from CellRank defined root cell
root_idx = np.where(adata.obs["ct_pseudotime"] == 0)[0][0]
adata.uns["iroot"] = root_idx

scv.pl.scatter(
    adata,
    color=["annotation", root_idx, "ct_pseudotime"],
    fontsize=16,
    cmap="viridis",
    perc=[2, 98],
    colorbar=True,
    rescale_color=[0, 1],
    title=["annotation", "root cell", "ct_pseudotime"],
)

In [ ]:
model = cr.ul.models.GAM(adata)

In [ ]:
cr.pl.gene_trends(
    adata,
    model=model,
    data_key="X",
    genes=["EREG", "S100A8", "S100A9",'S100A10','IL1RN','HLA-DQA2','PITX1','MBD2','HMGA1','STAT1','OVOL1','IRF6','ZNF750','KLF4'],
    ncols=1,
    time_key="ct_pseudotime",
    same_plot=True,
    hide_cells=True,
    figsize=(10, 70),
    n_test_points=200
)

In [ ]:
cr.pl.heatmap(
    adata,
    model,
    genes=adata.varm['terminal_lineage_drivers']["C01-E11-S100A8+APC_corr"].sort_values(ascending=False).index[:100],
    show_absorption_probabilities=False,
    lineages="C01-E11-S100A8+APC",
    backend="loky",
    time_key = "ct_pseudotime",
    figsize = (4, 6),
    save = 'heatmap.pdf'
)

In [ ]:
cr.pl.heatmap(
    adata,
    model,
    genes=adata.varm['terminal_lineage_drivers']["C01-E11-S100A8+APC_corr"].sort_values(ascending=False).index[:100],
    show_absorption_probabilities=True,
    lineages="C01-E11-S100A8+APC",
    backend="loky",
    time_key = "ct_pseudotime",
)